# UQCMF Multi-Probe + Neighbor Universe Effect

This Colab notebook runs the UQCMF model with Pantheon+SH0ES, BAO, and CMB data, including the Neighbor Universe Effect.

**Priors (limited):**
- $\Omega_m$: [0.2, 0.4]
- $h$: [0.65, 0.8]
- $\sigma_z$: [0.05, 0.5]
- $\beta_0$, $A_{\text{neighbor}}$, $z_c$: as previous ranges.

Upload these files before running:
- `Pantheon+SH0ES.dat`
- `BAO_data.txt`
- `CMB_data.txt`

In [ ]:
!pip install numpy scipy matplotlib emcee corner

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee, corner, zipfile, os
from scipy.integrate import quad
from scipy.stats import norm

In [ ]:
# Upload files manually in Colab before running this cell
pantheon_file = 'Pantheon+SH0ES.dat'
bao_file = 'BAO_data.txt'
cmb_file = 'CMB_data.txt'

# Load Pantheon+ data
z_sn, m_b, err = np.loadtxt(pantheon_file, usecols=(2,8,9), skiprows=1, unpack=True)

# Load BAO data
z_bao, dv_rd, dv_rd_err = np.loadtxt(bao_file, unpack=True)

# Load CMB data (shift parameters)
cmb_params = np.loadtxt(cmb_file, max_rows=1)
cmb_cov = np.loadtxt(cmb_file, skiprows=1)

In [ ]:
def E_z(z, Om, h):
    return np.sqrt(Om*(1+z)**3 + (1-Om))

def comoving_dist_scalar(z, Om, h):
    return quad(lambda zp: 1.0/E_z(zp, Om, h), 0, z)[0] * 2997.92458/h

def comoving_dist(z, Om, h):
    z = np.atleast_1d(z)
    return np.array([comoving_dist_scalar(zi, Om, h) for zi in z])

def luminosity_distance(z, Om, h):
    return (1+z) * comoving_dist(z, Om, h)

def distance_modulus(z, Om, h):
    return 5*np.log10(luminosity_distance(z, Om, h)*1e6/10)

def beta_mind_z(z, beta0, A_nbr, zc, sig_z):
    return beta0 + A_nbr * np.exp(-0.5*((z - zc)/sig_z)**2)

In [ ]:
def log_likelihood(theta):
    Om, h, beta0, A_nbr, zc, sig_z = theta
    mu_model = distance_modulus(z_sn, Om, h) + beta_mind_z(z_sn, beta0, A_nbr, zc, sig_z)
    ll_sn = -0.5 * np.sum(((m_b - mu_model)/err)**2)

    # BAO
    dv_model = (comoving_dist(z_bao, Om, h)**3 / z_bao)**(1/3)
    ll_bao = -0.5 * np.sum(((dv_model - dv_rd)/dv_rd_err)**2)

    # CMB
    Dc_1100 = comoving_dist(1100.0, Om, h).item()
    Dc_1100_silk = comoving_dist(1100.0, Om, h/np.sqrt(3)).item()
    la_model = np.pi * Dc_1100 / Dc_1100_silk
    R_model = np.sqrt(Om) * Dc_1100 * h/100
    zstar_model = 1100.0
    diff_cmb = np.array([la_model, R_model, zstar_model]) - cmb_params
    ll_cmb = -0.5 * diff_cmb @ np.linalg.inv(cmb_cov) @ diff_cmb

    return ll_sn + ll_bao + ll_cmb

In [ ]:
def log_prior(theta):
    Om, h, beta0, A_nbr, zc, sig_z = theta
    if not (0.2 <= Om <= 0.4):
        return -np.inf
    if not (0.65 <= h <= 0.8):
        return -np.inf
    if not (-1.0 <= beta0 <= 1.0):
        return -np.inf
    if not (-0.3 <= A_nbr <= 0.3):
        return -np.inf
    if not (0 <= zc <= 2):
        return -np.inf
    if not (0.05 <= sig_z <= 0.5):
        return -np.inf
    return 0.0

In [ ]:
def log_posterior(theta):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta)

In [ ]:
ndim = 6
nwalkers = 32
nsteps = 3000
p0 = np.array([0.3, 0.7, 0.0, 0.0, 0.5, 0.2]) + 1e-4*np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior)
sampler.run_mcmc(p0, nsteps, progress=True)

In [ ]:
flat_samples = sampler.get_chain(discard=500, thin=10, flat=True)
fig = corner.corner(flat_samples, labels=[r'$\Omega_m$', 'h', r'$\beta_0$', r'$A_{nbr}$', r'$z_c$', r'$\sigma_z$'],
                    truths=[None]*6)
fig.savefig('posteriors_neighbor_limited.png', dpi=200)

# Save summary
with open('fit_summary.txt', 'w') as f:
    for i, name in enumerate([r'$\Omega_m$', 'h', r'$\beta_0$', r'$A_{nbr}$', r'$z_c$', r'$\sigma_z$']):
        mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        f.write(f"{name} = {mcmc[1]:.5f} (+{q[1]:.5f}, -{q[0]:.5f})
")

# Zip outputs
with zipfile.ZipFile('UQCMF_results_neighbor_limitedpriors.zip', 'w') as zf:
    zf.write('posteriors_neighbor_limited.png')
    zf.write('fit_summary.txt')